In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
##################################################################################
##### Define all parameters for model tuning
##################################################################################

n_fold = 5
expName = "NT_Site_PredNTS_Classification_INT"
outPath = "Results"
foldName = "folds.pickle"

epochs = 100
batch_size = 64
shuffle = True
seed = None

input_data_folder = "Data"
training_data_file = "Training-datasets-PredNTS.txt"
independent_data_file = "independent dataset-PredNTS.txt"

In [4]:
import os 
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score, matthews_corrcoef

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.svm import SVC

import math

In [5]:
# print(tf.test.is_gpu_available(cuda_only=True))
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

2.8.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Utility Functions

In [6]:
##################################################################################
##### define all CUSTOM functions
##################################################################################

def integer_encode_nt(sequence, char_dict):
    
    seq_encoded = np.ones((len(sequence))) * -1
    
    i = 0
    for single_character in sequence:
        if(single_character.upper() in char_dict.keys()):
            seq_encoded[i] = char_dict[single_character.upper()]
            i = i+1
        else:
            raise ValueError('Incorrect character in NT sequence: '+sequence)
    return seq_encoded

In [7]:
##################################################################################
##### Build k-fold functions
##################################################################################

## Build the K-fold from dataset
def build_kfold(features, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features, labels):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X_train": X_train,
            "X_test": X_test,
            "y_train":y_train,
            "y_test":y_test
        })
        
    return kfoldList

In [8]:
##################################################################################
##### define evaluator functions
##################################################################################

def pred2label(y_pred):
    y_pred = np.round(y_pred)
    return y_pred

# Model

In [9]:
def get_model():
    
    model = RandomForestClassifier(n_estimators=100, 
                                   criterion='gini',
                                   max_depth = 10,
                                   max_features = 'sqrt',
                                   bootstrap=True,
                                   oob_score=True)
    
    return model

# Train Dataset

In [27]:
##################################################################################
##### read training file
##################################################################################
train_file_path = os.path.join(input_data_folder, training_data_file)
train_data = pd.read_csv(train_file_path, sep='\t', header=None)
train_data.columns = ['Sequence', 'name', 'id', 'flag', 'label_original', 'type']
train_data.head()

##################################################################################
##### Create dictionary of all characters in the NT sequence 
##################################################################################
all_char_set = set({})
for val in [set(val) for val in train_data['Sequence']]:
    all_char_set = all_char_set.union(val)
all_char_list = list(all_char_set)
all_char_list.sort()
all_char_dict = {}
for i in range(len(all_char_list)):
    all_char_dict[all_char_list[i]] = i
    
##################################################################################
##### Create OHE of sequence
##################################################################################
train_data['INT_Sequence'] = pd.Series([integer_encode_nt(val, all_char_dict) 
                                        for val in train_data["Sequence"]])

##################################################################################
##### Fix the labels
##################################################################################
train_data['label'] = pd.Series([1 if val == 1 else 0 
                                 for val in train_data["label_original"]])

##################################################################################
##### Extract features and labels, create folds
##################################################################################

train_features = np.array(list(train_data['INT_Sequence']))
# train_features = train_features[:, :, np.newaxis]
train_labels = np.array(list(train_data['label']))
# train_labels = train_labels[:, np.newaxis]

# Independent data

In [31]:
##################################################################################
##### read independent data file
##################################################################################
indpe_file_path = os.path.join(input_data_folder, independent_data_file)
indpe_data = pd.read_csv(indpe_file_path, sep='\t', header=None)
indpe_data.columns = ['Sequence', 'name', 'id', 'flag', 'label_original', 'type']
indpe_data.head()
    
##################################################################################
##### Create OHE of sequence
##################################################################################
indpe_data['INT_Sequence'] = pd.Series([integer_encode_nt(val, all_char_dict) 
                                        for val in indpe_data["Sequence"]])

##################################################################################
##### Fix the labels
##################################################################################
indpe_data['label'] = pd.Series([1 if val == 1 else 0 
                                 for val in indpe_data["label_original"]])

##################################################################################
##### Extract features and labels, create folds
##################################################################################

indpe_features = np.array(list(indpe_data['INT_Sequence']))
# indpe_features = indpe_features[:, :, np.newaxis]
indpe_labels = np.array(list(indpe_data['label']))
# indpe_labels = indpe_labels[:, np.newaxis]

input_seq_shape = indpe_features[0].shape

# Build folds using training features

In [32]:
folds = build_kfold(train_features, train_labels, k=n_fold, shuffle=shuffle, seed=seed)

## Write the k-fold dataset to file
foldPath = os.path.join(outPath, expName, "{}fold".format(n_fold))
if(not os.path.isdir(foldPath)):
    os.makedirs(foldPath)
pickle.dump(folds, open(os.path.join(foldPath, foldName), "wb"))

# Training Evaluation

In [70]:
##################################################################################
##### For each input file, train model and generate different outputs in a structured folder
##################################################################################

## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}
        
##################################################################################
##### extract data from the current fasta file
##################################################################################

print("\n======================================================================")
print("Training Positive:", np.sum(train_labels))
print("Training Negative:", train_labels.shape[0] - np.sum(train_labels))
print("Independent Positive:", np.sum(indpe_labels))
print("Independent Negative:", indpe_labels.shape[0] - np.sum(indpe_labels))

##################################################################################
##### TRAIN and PREDICT for every Fold, using models
##################################################################################

## Create and set directory to save model
modelPath = os.path.join(outPath, expName, "{}fold".format(n_fold), "models")
if(not os.path.isdir(modelPath)):
    os.makedirs(modelPath)

# fold counter
i = 0

for fold in folds:

    # adding random shuffling of the dataset for training purpose
    randomized_index_arr = np.arange(fold["X_train"].shape[0])
    randomized_index_arr = np.random.permutation(randomized_index_arr)

    print("\nTrain/Test model on Fold #"+str(i)+".")

    model_file_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))

    model = get_model()
    
#     ## Define the model callbacks for early stopping and saving the model. Then train model.
#     modelCallbacks = [
#         tf.keras.callbacks.ModelCheckpoint(model_file_path,
#                                            monitor = 'val_loss', verbose = 1, save_best_only = True, 
#                                            save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
#     ]
#     model.fit(x = fold["X_train"][randomized_index_arr], y = fold["y_train"][randomized_index_arr], 
#               batch_size = batch_size, epochs = epochs, verbose = 1, 
#               callbacks = modelCallbacks, validation_data = (fold["X_test"], fold["y_test"]))
    
#     model = tf.keras.models.load_model(current_model_path)

    model.fit(X = fold["X_train"][randomized_index_arr], y = fold["y_train"][randomized_index_arr])

    model_file_obj = open(model_file_path, 'wb')
    pickle.dump(model, model_file_obj)
    model_file_obj.close()

    ##################################################################################
    ##### Prediction and metrics for TRAIN dataset
    ##################################################################################

    y_pred = model.predict(fold["X_train"])
    label_pred = pred2label(y_pred)
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_train"], label_pred)
    prec = precision_score(fold["y_train"],label_pred)
    mcc = matthews_corrcoef(fold["y_train"], label_pred)

    conf = confusion_matrix(fold["y_train"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(fold["y_train"], y_pred)
    auc = roc_auc_score(fold["y_train"], y_pred)

    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Train")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)

    ##################################################################################
    ##### Prediction and metrics for TEST dataset
    ##################################################################################

    y_pred = model.predict(fold["X_test"])
    label_pred = pred2label(y_pred)
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_test"], label_pred)
    prec = precision_score(fold["y_test"],label_pred)
    mcc = matthews_corrcoef(fold["y_test"], label_pred)

    conf = confusion_matrix(fold["y_test"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(fold["y_test"], y_pred)
    auc = roc_auc_score(fold["y_test"], y_pred)

    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Test")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)

    i = i+1

##################################################################################
##### Independent Data performance
##################################################################################

print("\nIndependent evaluation for model.")

# adding random shuffling of the dataset for training purpose
randomized_index_arr = np.arange(train_features.shape[0])
randomized_index_arr = np.random.permutation(randomized_index_arr)

model_file_path = os.path.join(modelPath, "bestModel-full.hdf5")

# ## Define the model callbacks for early stopping and saving the model. Then train model.
# modelCallbacks = [
#     tf.keras.callbacks.ModelCheckpoint(model_file_path,
#                                        monitor = 'val_loss', verbose = 1, save_best_only = True, 
#                                        save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
# ]
# model.fit(x = train_enc_features[randomized_index_arr], y = train_labels[randomized_index_arr], 
#           batch_size = batch_size, epochs = epochs, verbose = 1, 
#           callbacks = modelCallbacks, validation_data = (indpe_enc_features, indpe_labels))

# model = tf.keras.models.load_model(current_model_path)

model = get_model()

model.fit(X = train_features[randomized_index_arr], y = train_labels[randomized_index_arr])

model_file_obj = open(model_file_path, 'wb')
pickle.dump(model, model_file_obj)
model_file_obj.close()

##################################################################################
##### Prediction and metrics for TEST dataset
##################################################################################

y_pred = model.predict(indpe_features)
label_pred = pred2label(y_pred)
# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
auc = roc_auc_score(indpe_labels, y_pred)

evaluations["Fold"].append(i)
evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################
##### Dump evaluations to a file
##################################################################################

evalPath = os.path.join(outPath, expName, "_Evaluation_All_Datasets")
if(not os.path.isdir(evalPath)):
    os.makedirs(evalPath)

pickle.dump(evaluations,
            open(os.path.join(evalPath, "{}fold_evaluations.pickle".format(n_fold)), "wb"))


Training Positive: 1191
Training Negative: 1191
Independent Positive: 203
Independent Negative: 1022

Train/Test model on Fold #0.

Train/Test model on Fold #1.

Train/Test model on Fold #2.

Train/Test model on Fold #3.

Train/Test model on Fold #4.

Independent evaluation for model.


In [71]:
evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.652245,0.214834,0.556701,0.413793,0.699609,0.090453
Test,0.597398,0.635524,0.597416,0.455937,0.738895,0.203205
Train,0.735936,0.815579,0.735932,0.610408,0.861456,0.487842


In [72]:
def get_model():
    
    model = RandomForestClassifier(n_estimators=100, 
                                   criterion='entropy',
                                   max_depth = 5,
                                   max_features = 'log2',
                                   bootstrap=True,
                                   oob_score=True)
    
    return model

In [26]:
# def get_model():
    
# #     model = RandomForestClassifier(n_estimators=10, 
# #                                    criterion='gini', 
# #                                    bootstrap=True,
# #                                    oob_score=True)

# #     model = xgb.XGBClassifier(objective="binary:logistic", use_label_encoder=False,
# #                               learn_rate=0.001, eta=0.1, eval_metric='auc'
# #                              )

#     model = xgb.XGBClassifier(objective="binary:logistic", 
#                               eval_metric='logloss',
#                               use_label_encoder=False, 
#                               max_depth=10,
#                               reg_lambda=0.001,
#                               learning_rate=0.001
#                              )
    
# #     model = SVC(kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, probability=False,
# #                 tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=100, decision_function_shape='ovr', 
# #                 break_ties=False, random_state=None)

# #     model = tf.keras.models.Sequential()
# #     model.add(tf.keras.layers.Input(shape=(latent_dim_size,)))
# #     model.add(tf.keras.layers.Dense(32,
# #                                     kernel_regularizer=tf.keras.regularizers.l2(0.001)))
# #     model.add(tf.keras.layers.Activation('relu'))
# #     model.add(tf.keras.layers.Dropout(0.5))
# #     model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
# #     model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.0001), 
# #                   loss='binary_crossentropy', 
# #                   metrics=None)
    
#     return model

In [27]:
from sklearn.decomposition import PCA, KernelPCA

In [28]:
pca = PCA(n_components=3)
pca.fit(np.concatenate((train_enc_features, indpe_enc_features)))
# pca.fit(indpe_enc_features)
print(sum(pca.explained_variance_ratio_), ' : ', pca.explained_variance_ratio_)

0.8622940182685852  :  [0.6662198  0.14358738 0.05248687]


In [29]:
pca.components_.shape

(3, 10)

In [30]:
pca.components_[0]

array([ 0.23529403, -0.04270929, -0.06038254,  0.05727866,  0.35982838,
       -0.32488906, -0.3668896 ,  0.3212422 , -0.35213375, -0.58228284],
      dtype=float32)

In [31]:
transformer = KernelPCA(n_components=7, kernel='rbf')
transformer.fit(np.concatenate((train_enc_features, indpe_enc_features)))
print(sum(transformer.explained_variance_ratio_), ' : ', transformer.explained_variance_ratio_)

AttributeError: 'KernelPCA' object has no attribute 'explained_variance_ratio_'